In [1]:
try: 
    import ee
except Exception:
    !pip install earthengine-api --upgrade    
finally:
    import ee    
    ee.Authenticate()
    ee.Initialize()

Enter verification code:  4/1AY0e-g4R8JGH7_6FeaLv7TISRmJRd0Pfu8Xy1uNhZFbvm4bPU2JK4c4lIZQ



Successfully saved authorization token.


In [2]:
from IPython.display import Image
import pprint
import matplotlib.pyplot as plt
import cudf as pd
import os

In [102]:
single_point = ee.Geometry.Point([-68.3182,19.1665])

dataset = ee.ImageCollection("HYCOM/sea_temp_salinity").filter(ee.Filter.date('2019-01-10', '2019-01-11')).filter(ee.Filter.geometry(single_point))
size = dataset.size().getInfo()
dataset = dataset.toList(size)
# print(ee.Image(dataset.get(0)).getInfo()['bands']['crs_transform'])

# image = ee.Image('HYCOM/sea_temp_salinity/2020030200').multiply(0.001).add(20)
image = ee.Image(dataset.get(0)).multiply(0.001).add(20)

image.reduceRegion(ee.Reducer.mean(), geometry=single_point).getInfo()

{'salinity_0': 35.763,
 'salinity_10': 35.765,
 'salinity_100': 36.432,
 'salinity_1000': 34.989000000000004,
 'salinity_12': 35.765,
 'salinity_125': 36.697,
 'salinity_1250': 34.995000000000005,
 'salinity_15': 35.766,
 'salinity_150': 36.855000000000004,
 'salinity_1500': 35.052,
 'salinity_2': 35.765,
 'salinity_20': 35.768,
 'salinity_200': 36.802,
 'salinity_2000': 35.064,
 'salinity_25': 35.775999999999996,
 'salinity_250': 36.622,
 'salinity_2500': 35.011,
 'salinity_30': 35.786,
 'salinity_300': 36.47,
 'salinity_3000': 34.972,
 'salinity_35': 35.799,
 'salinity_350': 36.329,
 'salinity_4': 35.765,
 'salinity_40': 35.815,
 'salinity_400': 36.144999999999996,
 'salinity_4000': 34.902,
 'salinity_45': 35.832,
 'salinity_50': 35.85,
 'salinity_500': 35.769999999999996,
 'salinity_5000': 34.902,
 'salinity_6': 35.765,
 'salinity_60': 35.914,
 'salinity_600': 35.476,
 'salinity_70': 36.037,
 'salinity_700': 35.206,
 'salinity_8': 35.765,
 'salinity_80': 36.184,
 'salinity_800': 35.

Load Equinox Data

In [3]:
RESULTS_PATH = '../../results/equinox'

df = pd.read_csv(os.path.join(RESULTS_PATH, 'df_latlon.csv'))
df.head()

,DATE_UTC,LAT,LONG
0,2019-01-10,19.1665,-68.3182
1,2019-01-10,19.1797,-68.3203
2,2019-01-10,19.1927,-68.3223
3,2019-01-10,19.2058,-68.3245
4,2019-01-10,19.2187,-68.3267


## Template/Documentation for basic Image Getting

In [ ]:
# Load an image.
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')

# All metadata.
print('All metadata:')
pprint.pprint(image.getInfo())

# Get information about the bands as a list.
band_names = image.bandNames()
print('Band names:', band_names.getInfo())  # ee.List of band names

# Get projection information from band 1.
b1_proj = image.select('B1').projection()
print('Band 1 projection:', b1_proj.getInfo())  # ee.Projection object

# Get scale (in meters) information from band 1.
b1_scale = image.select('B1').projection().nominalScale()
print('Band 1 scale:', b1_scale.getInfo())  # ee.Number

# Note that different bands can have different projections and scale.
b8_scale = image.select('B8').projection().nominalScale()
print('Band 8 scale:', b8_scale.getInfo())  # ee.Number

# Get a list of all metadata properties.
properties = image.propertyNames()
print('Metadata properties:',
      properties.getInfo())  # ee.List of metadata properties

# Get a specific metadata property.
cloudiness = image.get('CLOUD_COVER')
print('CLOUD_COVER:', cloudiness.getInfo())  # ee.Number

# Get version number (ingestion timestamp as microseconds since Unix epoch).
version = image.get('system:version')
print('Version:', version.getInfo())  # ee.Number
print('Version (as ingestion date):',
      ee.Date(ee.Number(version).divide(1000)).format().getInfo())  # ee.Date

# Get the timestamp.
ee_date = ee.Date(image.get('system:time_start'))
print('Timestamp:', ee_date.getInfo())  # ee.Date

# Date objects transferred to the client are milliseconds since UNIX epoch;
# convert to human readable date with ee.Date.format().
print('Datetime:', ee_date.format().getInfo())  # ISO standard date string

In [ ]:
import datetime
import csv
import cudf as pd
from tqdm import tqdm

ee.Initialize()

RESULTS_PATH = '../../results/equinox'

df = pd.read_csv(os.path.join(RESULTS_PATH, 'df_latlon.csv'))

IMAGE_COLLECTION = 'HYCOM/sea_temp_salinity'

file = open("../../data/HYCOM.csv", "a", newline='')
writer = csv.writer(file)

# write headers
writer.writerow(['start_date', 'lat', 'lon', 'water_temp_0', 'salinity_0' ,'water_temp_2','salinity_2','water_temp_4','salinity_4','water_temp_4' ,'salinity_4','water_temp_6', 'salinity_6', 'water_temp_8', 'salinity_8'])

data = []

for index, row in tqdm(df.to_pandas().iterrows(), desc=str([start_date, lat, lon, water_temp_0, salinity_0])):
    start_date = str(row['DATE_UTC'])
    end_date = (datetime.datetime.strptime(start_date, '%Y-%m-%d') + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    lat, lon = row['LAT'], row['LONG']
    point = ee.Geometry.Point([lon, lat])
    
    collection = ee.ImageCollection(IMAGE_COLLECTION).filter(ee.Filter.geometry(point)).filter(ee.Filter.date(start_date, end_date))
    size = collection.size().getInfo()    
    collection = collection.toList(size)
    try:
        for idx in range(size):

            
            image = ee.Image(collection.get(idx)).multiply(0.001).add(20)
            band_info = image.reduceRegion(ee.Reducer.mean(), geometry=point).getInfo()
            
            water_temp_0 = band_info['water_temp_0']
            salinity_0 = band_info['salinity_0']
            water_temp_2 =  band_info['water_temp_2']
            salinity_2 =  band_info['salinity_2']
            water_temp_4 =  band_info['water_temp_4']
            salinity_4 =  band_info['salinity_4']
            water_temp_6 =  band_info['water_temp_6']
            salinity_6 = band_info['salinity_6']
            water_temp_8 =  band_info['water_temp_8']
            salinity_8 =  band_info['salinity_8']               
            writer.writerow([start_date, lat, lon, water_temp_0, salinity_0 ,water_temp_2,salinity_2,water_temp_4,salinity_4,water_temp_4 ,salinity_4,water_temp_6, salinity_6, water_temp_8, salinity_8])
            print(str([start_date, lat, lon, water_temp_0, salinity_0]))
            
    except Exception as e:
        print(f'Error: {e}')
        continue

In [106]:
!xdg-open .